# Workshop: Textanalys

I dagens workshop skall ni arbeta i par för att utvärdera och problematisera förbehandlingens effekter på en modell. I vårt fall skall vi begränsa oss till att använda så kallade *temamodeller* eller hädanefter *topic models*. Dessa modellerar hur fördelningen av vissa termer hör ihop över dokument. 

Grundantagandet är att varje dokument av text är uppbyggt av ett antal teman, eller topics. Varje topics genererar ord associerat med sitt ämne, som i sin tur bygger upp själva dokumentet. En artikel i tidningen skulle kunna ha ett topic "sport" som genererar ord relaterade till cykling eller OS. Observera att detta är en statistisk *a priori* modell. Den egentliga processen är att det finns en reporter som skriver ner artikeln på order av sin arbetsgivare.

Tidigare i veckan diskuterade vi alltså den pipeline av förbehandling som text måste genomgå för att bli maskintolkbar. Under workshopen idag skall ni *i par* studera data från Riksdagens öppna data ([data.riksdagen.se](http://data.riksdagen.se/)) i syfte att studera vad som diskuteras i exempelvis motioner.

Workshopen kräver minimalt med programmering, annat än att kommentera bort och köra någon rad. Om ni har tidigare erfarenhet av python är det givetvis fritt fram att experimentera. Alla funktioner för workshopen återfinns i filen ``workshop.py``. 

In [2]:
import course.workshop
import nltk
from course.workshop import PunctuationRemover, LowerCaser, TokenizeCount, StemTokenizeCount, TopicModel
from sklearn.pipeline import Pipeline
import numpy as np
import pyLDAvis

nltk.download('stopwords')
np.random.seed(123)
pyLDAvis.enable_notebook()

[nltk_data] Downloading package stopwords to /home/vws/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Hämta datan
Datan hämtas med funktionen ``query_riksdagen`` som skickar en sökning (*query*) till riksdagens öppna API. Med detta kan vi göra en sökning på en viss term (exempelvis *skatt*, *terrorism*, eller *välfärd*) inom en viss typ av dokument (som *motioner*, *propositioner* och *EU-förslag*). Det finns även möjlighet att specificera tidsperioden. Default är att funktionen hämtar data från 2010-01-01 till 2020-01-01.

Argumentet ``topic`` anger sökordet i databasen. Argumentet ``doc_type`` anger dokumentationstypen, där ``mot`` anger motioner.

Det tar ett par sekunder att hämta datan.

In [3]:
data = course.workshop.query_riksdagen(topic='', doc_type='mot', limit=2)

Datan sparas i en lista (en typ av array), och ni kan inspektera motionernas textinnehåll genom index i listan.

In [3]:
data[3]

'Motion till riksdagen 2019/20:3019 av Jonas Sjöstedt m.fl. (V) Förslag till riksdagsbeslut Riksdagen anvisar anslagen för 2020 inom utgiftsområde 21 Energi enligt förslaget i tabell 1 i motionen. Anslagsfördelning Förutom de anslagsförändringar som redovisas i tabell \xa0 1 har vi inga avvikelser i förhållande till regeringens förslag. Tabell 1 Anslagsförslag 2020 för utgiftsområde 21 Energi Tusental kronor Ramanslag Regeringens förslag Avvikelse från regeringen (V) 1:3 Insatser för förnybar elproduktion 25 000 +50 000 1:5 Laddinfrastruktur längs större vägar 50 000 +50 000 Nytt Sol- och vindkraft offentliga byggnader  \xa0 + 75 0 000 Summa 3 468 932 +850 000 Anslag  1:3 Insatser för förnybar elproduktion Vi avvisar regeringens förslag om att sänka medlen till den förnybara elproduktionen genom  att slå ihop två tidigare anslag  och lägga en mindre summa än ti digare år. Vänsterpartiet föreslår en  ökning  av anslaget med 50 miljoner kronor jämfört med regeringen.  Anslag  1:5 Laddinf

Antalet dokument är längden på listan, vilket fås med funktionen ``len`` i python:

In [3]:
len(data)

40

## En minimal pipeline
En minimal pipeline behöver bestå av följande:

- Tokenisering
- Numerisk representation

Det är denna pipeline ni skall använda som baseline. Ni kommer därefter lägga till andra steg, så som rensning av punktuering och konvertering till gemener. Vi kommer använda oss av [scikit-learn](https://scikit-learn.org/stable/index.html), ett API för maskinlärning, som tillåter enkel konstruktion av reproducerbara pipelines. 

Funktionen ``CountVectorizer`` skapar ett objekt som **både** tokeniserar datan på ordnivå och skapar en frekvensrepresentation av datan. Vi behöver alltså bara denna funktion för en fungerande minimal pipeline:

In [4]:
# En minimal pipeline
TokenizeCount_w_stopwords = TokenizeCount()

pipeline = Pipeline([('tokenisera och räkna', TokenizeCount_w_stopwords)])

## Sammansatta pipelines

Vi lägger enkelt till fler inledande steg i pipelinen som en lista (som separeras med komma-tecken). Alla steg måste ha ett namn. Det sista steget är som sagt obligatoriskt.

Om ni vill utesluta ett steg kan ni *kommentera bort det* genom att lägga till ``#`` framför.

Paketet ``workshop.py`` innehåller ett antal bekvämlighetsfunktioner, som ni under era experiment skall lägga till och ta bort.

In [12]:
pipeline = Pipeline([
    ('ta bort punktuering', PunctuationRemover),
    ('gör om till gemener', LowerCaser),
    ('tokenisera och räkna', TokenizeCount_w_stopwords)
])

# Stopp-ord
Stopp-ord måste tas bort när vi väl har tokeniserat dokumenten. Vi skickar det därför som argument till vår tokeniserare.

Ni kan också lägga till egna stopp-ord genom att utöka listan.

In [3]:
swedish_stop_words = nltk.corpus.stopwords.words('swedish')

# Lägg till egna stoppord i listan
custom_stop_words = ['2016', 'ost', 'en']

swedish_stop_words.extend(custom_stop_words)

Lägg till stopp-orden i er pipeline:

In [4]:
# En sammansatt pipeline
TokenizeCount_wo_stopwords = TokenizeCount(stop_words=swedish_stop_words)

pipeline = Pipeline([
    ('ta bort punktuering', PunctuationRemover),
    ('gör om till gemener', LowerCaser),
    ('tokenisera och räkna', TokenizeCount_wo_stopwords)
])

## Stemming
Tyvärr finns det inte enkelt tillgängligt stöd för lemmatisering på svenska som öppen källkod. Därför kommer vi istället att använda stemming, det vill säga att man endast bevarar stammen hos ett ord. 



In [16]:
# En sammansatt pipeline med stemming
StemTokenizeCount_wo_stopwords = StemTokenizeCount(stop_words=swedish_stop_words)

pipeline = Pipeline([
    ('ta bort punktuering', PunctuationRemover),
    ('gör om till gemener', LowerCaser),
    ('tokenisera och räkna', StemTokenizeCount_wo_stopwords)
])

# Uppgifter

Under labbens gång skall ni genomföra följande uppgifter:

1. Ladda ner data från riksdagens öppna data. Ni kan utforska motioner via [riksdagsmotioner.dh.gu.se](https://riksdagsmotioner.dh.gu.se/).  
    - Jag rekommenderar att ni använder någon av följande dokumenttyper: 
        - ``kom``: EU-förslag
        - ``mot``: motion
        - ``prop``: proposition

2. Generera topics med LDA. Undersök kvaliteten på era topics med en minimal pipeline.
    - Vad får ni för topics? 
    - Vilka tokens förekommer oftast? 
    - Hur väl separerade är era topics?
    
3. Lägg till borttagning av punktuering i er pipeline. Hur påverkas resultaten i er topic model?
4. Lägg till gemenisering i er pipeline. Vad händer med era topics?
5. Slutligen, jämför resultatet med att byta ut ert sista tokeniseringssteg med en vektormodell.
6. Diskutera och förbered era resultat för redovisning.
7. Varje grupp kommer få presentera en deluppgift, med övriga grupper i diskussion.

## Slutgiltig pipeline
En slutgiltig pipeline ni kan arbeta med ser alltså ut så här:

In [18]:
swedish_stop_words = nltk.corpus.stopwords.words('swedish')

custom_stop_words = ['2016', 'ost', 'en']
swedish_stop_words.extend(custom_stop_words)

# Tokenisering utan filtrering av stopp-ord
TokenizeCount_w_stopwords = TokenizeCount()

# Tokenisering med filtrering av stopp-ord 
TokenizeCount_wo_stopwords = TokenizeCount(stop_words=swedish_stop_words)

# Tokenisering med stemming och med filtrering av stopport
StemTokenizeCount_wo_stopwords = StemTokenizeCount(stop_words=swedish_stop_words)

pipeline = Pipeline([
#     ('ta bort punktuering', PunctuationRemover),
#     ('gör om till gemener', LowerCaser),
    ('tokenisera och räkna', TokenizeCount_w_stopwords) # Minimal pipeline. Lägg till steg innan eller byt ut tokeniseringen.
])

In [ ]:
data = course.workshop.query_riksdagen(topic='', doc_type='mot', limit=2)

Resultaten fås genom att skicka resultaten till vår topic model:

In [19]:
TopicModel(pipeline, data, n_topics=10, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.168707  0.081491       1        1  30.202233
1      0.063636  0.197880       2        1  25.476605
5     -0.084294  0.168404       3        1  20.981150
7      0.158740  0.096103       4        1  14.458008
0      0.011058  0.051544       5        1   3.873030
6     -0.107807 -0.034160       6        1   2.579991
4      0.149355 -0.091697       7        1   2.416051
9     -0.007327 -0.156522       8        1   0.004311
8     -0.007327 -0.156522       9        1   0.004311
3     -0.007327 -0.156522      10        1   0.004311, topic_info=                      Term         Freq        Total Category  logprob  \
10573           regeringen   786.000000   786.000000  Default  30.0000   
12671              sverige   554.000000   554.000000  Default  29.0000   
11582                  ska  1086.000000  1086.000000  Default  28.0000   
2210                   bör   712.000000   712.000000  Default  27.0000   
4270               företag   146.000000   146.000000  Default  26.0000   
...                    ...          ...          ...      ...      ...   
165                 203019     0.000251     1.482558  Topic10  -9.6512   
680         anslagsförslag     0.000251     1.482558  Topic10  -9.6512   
681    anslagsförändringar     0.000251     1.482558  Topic10  -9.6512   
308                    750     0.000251     1.482558  Topic10  -9.6512   
6669           klimatpaket     0.000251     1.482558  Topic10  -9.6512   

       loglift  
10573  30.0000  
12671  29.0000  
11582  28.0000  
2210   27.0000  
4270   26.0000  
...        ...  
165     1.3668  
680     1.3668  
681     1.3668  
308     1.3668  
6669    1.3668  

[791 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
24         4  0.840080              1325
37         7  0.865154               169
109        7  0.875081            200940
113        6  0.863086           2010110
119        6  0.863086             20141
...      ...       ...               ...
15362      4  0.150372               öka
15362      5  0.030074               öka
15362      6  0.006015               öka
15365      7  0.875081            ökades
15472      7  0.861895  överklaganderätt

[1237 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 6, 8, 1, 7, 5, 10, 9, 4])

In [15]:
data[0]

'Motion till riksdagen 2019/20:3351 av Johan Pehrson m.fl. (L) Förslag till riksdagsbeslut Riksdagen ställer sig bakom det som anförs i motionen om principen om att förorenaren ska betala och tillkännager detta för regeringen. Riksdagen ställer sig bakom det som anförs i motionen om  att  mer EU-samarbete krävs för klimatets skull och tillkännager detta för regeringen. Riksdagen ställer sig bakom det som anförs i motionen om en europeisk koldioxidskatt och tillkännager detta för regeringen. Riksdagen ställer sig bakom det som anförs i motionen om vikten av mer forskning och utveckling för att lösa klimatkrisen och tillkännager detta för regeringen. Riksdagen ställer sig bakom det som anförs i motionen om konsumenters val och tillkännager detta för regeringen. Riksdagen ställer sig bakom det som anförs i motionen om att bryta beroendet av fossila bränslen och tillkännager detta för regeringen. Riksdagen ställer sig bakom det som anförs i motionen om biodrivmedel och tillkännager detta f

In [8]:
pipeline.fit_transform(data)

<40x16564 sparse matrix of type '<class 'numpy.int64'>'
	with 46638 stored elements in Compressed Sparse Row format>